# HW4 2021300003004 张天骏

## benchmark model

In [7]:
# import

import timeit
import numpy as np
import matplotlib.pyplot as plt
beta = 0.95
alpha = 1 / 3
gamma = 1.5
delta = 0.1
kss = ((1 - beta * (1-delta)) / (alpha * beta)) ** (1 / (alpha - 1))
kmin = 0.5 * kss
kmax = 1.5 * kss
n=1000
kgrid = np.linspace(kmin, kmax, n)

In [3]:
# step 1
V0 = np.zeros(n)
# step 2
def V_update(V):
    global gamma
    global delta
    V_new = np.zeros(n)  # initialize updated value function and policy function
    g_new = np.zeros(n)
    for i in range(n):  # loop over grid points
        k = kgrid[i]
        k_bound = k**alpha+(1-delta)*k  # keep non-zero consumption
        V_max = -np.inf  # initialize maximum value
        for j in range(n):  # loop over possible k_next
            k_next = kgrid[j]
            V_next = V[j]
            if k_next < k_bound:  # check feasibility
                c = k**alpha - k_next + (1-delta)*k
                V_current = (c**(1-gamma))/(1-gamma)+beta*V_next
                if V_current > V_max:
                    V_max = V_current
                    g_k = k_next
        V_new[i] = V_max  # save
        g_new[i] = g_k
    return V_new, g_new

In [4]:
# step 3
def V_iteration(V_initial,tol):
    V = V_initial
    error = np.inf
    count = 0
    max_iter = 1000
    print_skip = 50
    while count < max_iter and error > tol:
        V_new, g_new = V_update(V)
        error = np.max(np.abs(V_new - V))
        V = V_new
        count = count + 1
        if count % print_skip == 0:
            print(f"Error at iteration {count} is {error}.")
    if error > tol:
        print("Failed to converge!")
    else:
        print(f"\nConverged in {count} iterations.")
    return V_new, g_new

In [5]:
start_time = timeit.default_timer()
V, g = V_iteration(V0,tol=10e-6)
print("The time difference is :", timeit.default_timer() - start_time)

Error at iteration 50 is 0.15562739536149195.
Error at iteration 100 is 0.011973136388760963.
Error at iteration 150 is 0.0009212726834419982.
Error at iteration 200 is 7.088730387039277e-05.

Converged in 239 iterations.
The time difference is : 286.1996236


## 1(a) concave value function only

由于价值函数$V(\cdot)$是凹的，每次更新得到的最优价值函数唯一，从而一旦发现价值函数下降，便可以停止搜索，以当前值作为更新结果。代码如下：

In [18]:
# step 1
V0 = np.zeros(n)
# step 2
def V_update(V):
    global gamma
    global delta
    global V_max
    V_new = np.zeros(n)  # initialize updated value function and policy function
    g_new = np.zeros(n)
    for i in range(n):  # loop over grid points
        k = kgrid[i]
        k_bound = k**alpha+(1-delta)*k  # keep non-zero consumption
        V_max = -np.inf  # initialize maximum value
        g_k = kss  # initialize g_k
        for j in range(n):  # loop over possible k_next
            k_next = kgrid[j]
            V_next = V[j]
            if k_next < k_bound:  # check feasibility
                c = k**alpha - k_next + (1-delta)*k
                V_current = (c**(1-gamma))/(1-gamma)+beta*V_next
                if V_current > V_max:
                    V_max = V_current
                    g_k = k_next
                else:
                    break
        V_new[i] = V_max  # save
        g_new[i] = g_k
    return V_new, g_new

In [19]:
# step 3
def V_iteration(V_initial,tol):
    V = V_initial
    error = np.inf
    count = 0
    max_iter = 1000
    print_skip = 50
    while count < max_iter and error > tol:
        V_new, g_new = V_update(V)
        error = np.max(np.abs(V_new - V))
        V = V_new
        count = count + 1
        if count % print_skip == 0:
            print(f"Error at iteration {count} is {error}.")
    if error > tol:
        print("Failed to converge!")
    else:
        print(f"\nConverged in {count} iterations.")
    return V_new, g_new

In [20]:
start_time = timeit.default_timer()
V, g = V_iteration(V0,tol=10e-6)
print("The time difference is :", timeit.default_timer() - start_time)

Error at iteration 50 is 0.15562739536149195.
Error at iteration 100 is 0.011973136388760963.
Error at iteration 150 is 0.0009212726834419982.
Error at iteration 200 is 7.088730387039277e-05.

Converged in 239 iterations.
The time difference is : 179.71150850000004


## 1(b) increasing policy function only

由于对策函数$g(k)$是单增的，所以第$t$次搜索的起点可以从$g(k_{t-1})$开始，这样做可以显著减少运算次数。代码如下：

In [77]:
# step 1
V0 = np.zeros(n)
# step 2
def V_update(V):
    global gamma
    global delta
    V_new = np.zeros(n)  # initialize updated value function and policy function
    g_new = np.zeros(n)
    k_start = 0
    for i in range(n):  # loop over grid points
        k = kgrid[i]
        k_bound = k**alpha+(1-delta)*k  # keep non-zero consumption
        V_max = -np.inf  # initialize maximum value
        for j in range(k_start,n):  # loop over possible k_next
            k_next = kgrid[j]
            V_next = V[j]
            if k_next < k_bound:  # check feasibility
                c = k**alpha - k_next + (1-delta)*k
                V_current = (c**(1-gamma))/(1-gamma)+beta*V_next
                if V_current > V_max:
                    V_max = V_current
                    g_k = k_next
                    k_start = j
        V_new[i] = V_max  # save
        g_new[i] = g_k
    return V_new, g_new

In [78]:
# step 3
def V_iteration(V_initial,tol):
    V = V_initial
    error = np.inf
    count = 0
    max_iter = 1000
    print_skip = 50
    while count < max_iter and error > tol:
        V_new, g_new = V_update(V)
        error = np.max(np.abs(V_new - V))
        V = V_new
        count = count + 1
        if count % print_skip == 0:
            print(f"Error at iteration {count} is {error}.")
    if error > tol:
        print("Failed to converge!")
    else:
        print(f"\nConverged in {count} iterations.")
    return V_new, g_new

In [79]:
start_time = timeit.default_timer()
V, g = V_iteration(V0,tol=10e-6)
print("The time difference is :", timeit.default_timer() - start_time)

Error at iteration 50 is 0.15562739536149195.
Error at iteration 100 is 0.011973136388760963.
Error at iteration 150 is 0.0009212726834419982.
Error at iteration 200 is 7.088730387039277e-05.

Converged in 239 iterations.
The time difference is : 124.27733249999983


## 1(c) both concave value function and increasing policy function

下面我将1(a)(b)中的操作合并，得到如下代码。可以发现运算时间实现了极为显著的降低：

In [131]:
# step 1
V0 = np.zeros(n)
# step 2
def V_update(V):
    global gamma
    global delta
    V_new = np.zeros(n)  # initialize updated value function and policy function
    g_new = np.zeros(n)
    k_start = 0
    for i in range(n):  # loop over grid points
        k = kgrid[i]
        k_bound = k**alpha+(1-delta)*k  # keep non-zero consumption
        V_max = -np.inf  # initialize maximum value
        g_k = kss  # initialize g_k
        for j in range(k_start,n):  # loop over possible k_next
            k_next = kgrid[j]
            V_next = V[j]
            if k_next < k_bound:  # check feasibility
                c = k**alpha - k_next + (1-delta)*k
                V_current = (c**(1-gamma))/(1-gamma)+beta*V_next
                if V_current > V_max:
                    V_max = V_current
                    g_k = k_next
                    k_start=j
                else:
                    break
        V_new[i] = V_max  # save
        g_new[i] = g_k
    return V_new, g_new

In [132]:
# step 3
def V_iteration(V_initial,tol):
    V = V_initial
    error = np.inf
    count = 0
    max_iter = 1000
    print_skip = 50
    while count < max_iter and error > tol:
        V_new, g_new = V_update(V)
        error = np.max(np.abs(V_new - V))
        V = V_new
        count = count + 1
        if count % print_skip == 0:
            print(f"Error at iteration {count} is {error}.")
    if error > tol:
        print("Failed to converge!")
    else:
        print(f"\nConverged in {count} iterations.")
    return V_new, g_new

In [133]:
start_time = timeit.default_timer()
V, g = V_iteration(V0,tol=10e-6)
print("The time difference is :", timeit.default_timer() - start_time)

Error at iteration 50 is 0.15562739536149195.
Error at iteration 100 is 0.011973136388760963.
Error at iteration 150 is 0.0009212726834419982.
Error at iteration 200 is 7.088730387039277e-05.

Converged in 239 iterations.
The time difference is : 1.2397132000005513


## 2 Multigrid

按照Multigrid的思路，我现在取100点的网格下得到了一个“稀疏收敛”的价值函数：

In [191]:
# first search
beta = 0.95
alpha = 1 / 3
gamma = 1.5
delta = 0.1
kss = ((1 - beta * (1-delta)) / (alpha * beta)) ** (1 / (alpha - 1))
kmin = 0.5 * kss
kmax = 1.5 * kss
n=100
kgrid = np.linspace(kmin, kmax, n)
# step 1
V0 = np.zeros(n)
# step 2
def V_update(V):
    global gamma
    global delta
    V_new = np.zeros(n)  # initialize updated value function and policy function
    g_new = np.zeros(n)
    for i in range(n):  # loop over grid points
        k = kgrid[i]
        k_bound = k**alpha+(1-delta)*k  # keep non-zero consumption
        V_max = -np.inf  # initialize maximum value
        for j in range(n):  # loop over possible k_next
            k_next = kgrid[j]
            V_next = V[j]
            if k_next < k_bound:  # check feasibility
                c = k**alpha - k_next + (1-delta)*k
                V_current = (c**(1-gamma))/(1-gamma)+beta*V_next
                if V_current > V_max:
                    V_max = V_current
                    g_k = k_next
        V_new[i] = V_max  # save
        g_new[i] = g_k
    return V_new, g_new
# step 3
def V_iteration(V_initial,tol):
    V = V_initial
    error = np.inf
    count = 0
    max_iter = 1000
    print_skip = 50
    while count < max_iter and error > tol:
        V_new, g_new = V_update(V)
        error = np.max(np.abs(V_new - V))
        V = V_new
        count = count + 1
        if count % print_skip == 0:
            print(f"Error at iteration {count} is {error}.")
    if error > tol:
        print("Failed to converge!")
    else:
        print(f"\nConverged in {count} iterations.")
    return V_new, g_new
start_time = timeit.default_timer()
V, g = V_iteration(V0,tol=10e-6)
print("The time difference is :", timeit.default_timer() - start_time)

Error at iteration 50 is 0.15564094679169926.
Error at iteration 100 is 0.011973225541353827.
Error at iteration 150 is 0.0009212795432702592.
Error at iteration 200 is 7.088783168285318e-05.

Converged in 239 iterations.
The time difference is : 2.9914582000001246


然后，我加密网格到1000个取点，并将上面得到的价值函数通过线性插值的方式加密到1000个取点，实现和密网格的对应。然后我以插值后的价值函数作为迭代的初始条件，显著减少了迭代的次数，从而减少了运算时间

In [192]:
# second search
beta = 0.95
alpha = 1 / 3
gamma = 1.5
delta = 0.1
kss = ((1 - beta * (1-delta)) / (alpha * beta)) ** (1 / (alpha - 1))
kmax = 1.5 * kss
kmin = 0.5 * kss
n=1000
kgrid = np.linspace(kmin, kmax, n)
# transform V0
from scipy.optimize import minimize_scalar
V0=np.interp(kgrid,np.linspace(kmin, kmax, 100),V)
# step 2
def V_update(V):
    global gamma
    global delta
    V_new = np.zeros(n)  # initialize updated value function and policy function
    g_new = np.zeros(n)
    for i in range(n):  # loop over grid points
        k = kgrid[i]
        k_bound = k**alpha+(1-delta)*k  # keep non-zero consumption
        V_max = -np.inf  # initialize maximum value
        for j in range(n):  # loop over possible k_next
            k_next = kgrid[j]
            V_next = V[j]
            if k_next < k_bound:  # check feasibility
                c = k**alpha - k_next + (1-delta)*k
                V_current = (c**(1-gamma))/(1-gamma)+beta*V_next
                if V_current > V_max:
                    V_max = V_current
                    g_k = k_next
        V_new[i] = V_max  # save
        g_new[i] = g_k
    return V_new, g_new
# step 3
def V_iteration(V_initial,tol):
    V = V_initial
    error = np.inf
    count = 0
    max_iter = 1000
    print_skip = 50
    while count < max_iter and error > tol:
        V_new, g_new = V_update(V)
        error = np.max(np.abs(V_new - V))
        V = V_new
        count = count + 1
        if count % print_skip == 0:
            print(f"Error at iteration {count} is {error}.")
    if error > tol:
        print("Failed to converge!")
    else:
        print(f"\nConverged in {count} iterations.")
    return V_new, g_new
start_time = timeit.default_timer()
V, g = V_iteration(V0,tol=10e-6)
print("The time difference is :", timeit.default_timer() - start_time)


Converged in 31 iterations.
The time difference is : 43.685797699999966


## 3 numba acceleration

我现在用numba包中的njit修饰代码中的$V_{update}(\cdot)$函数，从编译角度可以实现运行速度的提升

### initial code

In [9]:
from numba import njit

In [12]:
# import
import timeit
import numpy as np
import matplotlib.pyplot as plt
beta = 0.95
alpha = 1 / 3
gamma = 1.5
delta = 0.1
kss = ((1 - beta * (1-delta)) / (alpha * beta)) ** (1 / (alpha - 1))
kmin = 0.5 * kss
kmax = 1.5 * kss
n=1000
kgrid = np.linspace(kmin, kmax, n)
#step 1
V0 = np.zeros(n)
# step 2
@njit
def V_update(V):
    global gamma
    global delta
    V_new = np.zeros(n)  # initialize updated value function and policy function
    g_new = np.zeros(n)
    for i in range(n):  # loop over grid points
        k = kgrid[i]
        k_bound = k**alpha+(1-delta)*k  # keep non-zero consumption
        V_max = -np.inf  # initialize maximum value
        for j in range(n):  # loop over possible k_next
            k_next = kgrid[j]
            V_next = V[j]
            if k_next < k_bound:  # check feasibility
                c = k**alpha - k_next + (1-delta)*k
                V_current = (c**(1-gamma))/(1-gamma)+beta*V_next
                if V_current > V_max:
                    V_max = V_current
                    g_k = k_next
        V_new[i] = V_max  # save
        g_new[i] = g_k
    return V_new, g_new
# step 3
def V_iteration(V_initial,tol):
    V = V_initial
    error = np.inf
    count = 0
    max_iter = 1000
    print_skip = 50
    while count < max_iter and error > tol:
        V_new, g_new = V_update(V)
        error = np.max(np.abs(V_new - V))
        V = V_new
        count = count + 1
        if count % print_skip == 0:
            print(f"Error at iteration {count} is {error}.")
    if error > tol:
        print("Failed to converge!")
    else:
        print(f"\nConverged in {count} iterations.")
    return V_new, g_new
start_time = timeit.default_timer()
V, g = V_iteration(V0,tol=10e-6)
print("The time difference is :", timeit.default_timer() - start_time)

Error at iteration 50 is 0.15562739536149195.
Error at iteration 100 is 0.011973136388760963.
Error at iteration 150 is 0.0009212726834419982.
Error at iteration 200 is 7.088730387039277e-05.

Converged in 239 iterations.
The time difference is : 6.652758100000028


### 1(a)

In [16]:
# import
import timeit
import numpy as np
import matplotlib.pyplot as plt
beta = 0.95
alpha = 1 / 3
gamma = 1.5
delta = 0.1
kss = ((1 - beta * (1-delta)) / (alpha * beta)) ** (1 / (alpha - 1))
kmin = 0.5 * kss
kmax = 1.5 * kss
n=1000
kgrid = np.linspace(kmin, kmax, n)
#step 1
V0 = np.zeros(n)
# step 2
@njit
def V_update(V):
    global gamma
    global delta
    V_new = np.zeros(n)  # initialize updated value function and policy function
    g_new = np.zeros(n)
    for i in range(n):  # loop over grid points
        k = kgrid[i]
        k_bound = k**alpha+(1-delta)*k  # keep non-zero consumption
        V_max = -np.inf  # initialize maximum value
        for j in range(n):  # loop over possible k_next
            k_next = kgrid[j]
            V_next = V[j]
            if k_next < k_bound:  # check feasibility
                c = k**alpha - k_next + (1-delta)*k
                V_current = (c**(1-gamma))/(1-gamma)+beta*V_next
                if V_current > V_max:
                    V_max = V_current
                    g_k = k_next
                else:break
        V_new[i] = V_max  # save
        g_new[i] = g_k
    return V_new, g_new
# step 3
def V_iteration(V_initial,tol):
    V = V_initial
    error = np.inf
    count = 0
    max_iter = 1000
    print_skip = 50
    while count < max_iter and error > tol:
        V_new, g_new = V_update(V)
        error = np.max(np.abs(V_new - V))
        V = V_new
        count = count + 1
        if count % print_skip == 0:
            print(f"Error at iteration {count} is {error}.")
    if error > tol:
        print("Failed to converge!")
    else:
        print(f"\nConverged in {count} iterations.")
    return V_new, g_new
start_time = timeit.default_timer()
V, g = V_iteration(V0,tol=10e-6)
print("The time difference is :", timeit.default_timer() - start_time)

Error at iteration 50 is 0.15562739536149195.
Error at iteration 100 is 0.011973136388760963.
Error at iteration 150 is 0.0009212726834419982.
Error at iteration 200 is 7.088730387039277e-05.

Converged in 239 iterations.
The time difference is : 4.2079571999997825


### 1(b)

In [15]:
# import
import timeit
import numpy as np
import matplotlib.pyplot as plt
beta = 0.95
alpha = 1 / 3
gamma = 1.5
delta = 0.1
kss = ((1 - beta * (1-delta)) / (alpha * beta)) ** (1 / (alpha - 1))
kmin = 0.5 * kss
kmax = 1.5 * kss
n=1000
kgrid = np.linspace(kmin, kmax, n)
# step 1
V0 = np.zeros(n)
# step 2
@njit
def V_update(V):
    global gamma
    global delta
    V_new = np.zeros(n)  # initialize updated value function and policy function
    g_new = np.zeros(n)
    k_start = 0
    for i in range(n):  # loop over grid points
        k = kgrid[i]
        k_bound = k**alpha+(1-delta)*k  # keep non-zero consumption
        V_max = -np.inf  # initialize maximum value
        for j in range(k_start,n):  # loop over possible k_next
            k_next = kgrid[j]
            V_next = V[j]
            if k_next < k_bound:  # check feasibility
                c = k**alpha - k_next + (1-delta)*k
                V_current = (c**(1-gamma))/(1-gamma)+beta*V_next
                if V_current > V_max:
                    V_max = V_current
                    g_k = k_next
                    k_start = j
        V_new[i] = V_max  # save
        g_new[i] = g_k
    return V_new, g_new
# step 3
def V_iteration(V_initial,tol):
    V = V_initial
    error = np.inf
    count = 0
    max_iter = 1000
    print_skip = 50
    while count < max_iter and error > tol:
        V_new, g_new = V_update(V)
        error = np.max(np.abs(V_new - V))
        V = V_new
        count = count + 1
        if count % print_skip == 0:
            print(f"Error at iteration {count} is {error}.")
    if error > tol:
        print("Failed to converge!")
    else:
        print(f"\nConverged in {count} iterations.")
    return V_new, g_new
start_time = timeit.default_timer()
V, g = V_iteration(V0,tol=10e-6)
print("The time difference is :", timeit.default_timer() - start_time)

Error at iteration 50 is 0.15562739536149195.
Error at iteration 100 is 0.011973136388760963.
Error at iteration 150 is 0.0009212726834419982.
Error at iteration 200 is 7.088730387039277e-05.

Converged in 239 iterations.
The time difference is : 2.6339558000001944


### 1(c)

In [17]:
# import
import timeit
import numpy as np
import matplotlib.pyplot as plt
beta = 0.95
alpha = 1 / 3
gamma = 1.5
delta = 0.1
kss = ((1 - beta * (1-delta)) / (alpha * beta)) ** (1 / (alpha - 1))
kmin = 0.5 * kss
kmax = 1.5 * kss
n=1000
kgrid = np.linspace(kmin, kmax, n)
# step 1
V0 = np.zeros(n)
# step 2
@njit
def V_update(V):
    global gamma
    global delta
    V_new = np.zeros(n)  # initialize updated value function and policy function
    g_new = np.zeros(n)
    k_start = 0
    for i in range(n):  # loop over grid points
        k = kgrid[i]
        k_bound = k**alpha+(1-delta)*k  # keep non-zero consumption
        V_max = -np.inf  # initialize maximum value
        g_k = kss  # initialize g_k
        for j in range(k_start,n):  # loop over possible k_next
            k_next = kgrid[j]
            V_next = V[j]
            if k_next < k_bound:  # check feasibility
                c = k**alpha - k_next + (1-delta)*k
                V_current = (c**(1-gamma))/(1-gamma)+beta*V_next
                if V_current > V_max:
                    V_max = V_current
                    g_k = k_next
                    k_start=j
                else:
                    break
        V_new[i] = V_max  # save
        g_new[i] = g_k
    return V_new, g_new
# step 3
def V_iteration(V_initial,tol):
    V = V_initial
    error = np.inf
    count = 0
    max_iter = 1000
    print_skip = 50
    while count < max_iter and error > tol:
        V_new, g_new = V_update(V)
        error = np.max(np.abs(V_new - V))
        V = V_new
        count = count + 1
        if count % print_skip == 0:
            print(f"Error at iteration {count} is {error}.")
    if error > tol:
        print("Failed to converge!")
    else:
        print(f"\nConverged in {count} iterations.")
    return V_new, g_new
start_time = timeit.default_timer()
V, g = V_iteration(V0,tol=10e-6)
print("The time difference is :", timeit.default_timer() - start_time)

Error at iteration 50 is 0.15562739536149195.
Error at iteration 100 is 0.011973136388760963.
Error at iteration 150 is 0.0009212726834419982.
Error at iteration 200 is 7.088730387039277e-05.

Converged in 239 iterations.
The time difference is : 0.37810940000008486


### 2

In [18]:
# first search
import timeit
import numpy as np
import matplotlib.pyplot as plt
beta = 0.95
alpha = 1 / 3
gamma = 1.5
delta = 0.1
kss = ((1 - beta * (1-delta)) / (alpha * beta)) ** (1 / (alpha - 1))
kmin = 0.5 * kss
kmax = 1.5 * kss
n=100
kgrid = np.linspace(kmin, kmax, n)
# step 1
V0 = np.zeros(n)
# step 2
@njit
def V_update(V):
    global gamma
    global delta
    V_new = np.zeros(n)  # initialize updated value function and policy function
    g_new = np.zeros(n)
    for i in range(n):  # loop over grid points
        k = kgrid[i]
        k_bound = k**alpha+(1-delta)*k  # keep non-zero consumption
        V_max = -np.inf  # initialize maximum value
        for j in range(n):  # loop over possible k_next
            k_next = kgrid[j]
            V_next = V[j]
            if k_next < k_bound:  # check feasibility
                c = k**alpha - k_next + (1-delta)*k
                V_current = (c**(1-gamma))/(1-gamma)+beta*V_next
                if V_current > V_max:
                    V_max = V_current
                    g_k = k_next
        V_new[i] = V_max  # save
        g_new[i] = g_k
    return V_new, g_new
# step 3
def V_iteration(V_initial,tol):
    V = V_initial
    error = np.inf
    count = 0
    max_iter = 1000
    print_skip = 50
    while count < max_iter and error > tol:
        V_new, g_new = V_update(V)
        error = np.max(np.abs(V_new - V))
        V = V_new
        count = count + 1
        if count % print_skip == 0:
            print(f"Error at iteration {count} is {error}.")
    if error > tol:
        print("Failed to converge!")
    else:
        print(f"\nConverged in {count} iterations.")
    return V_new, g_new
start_time = timeit.default_timer()
V, g = V_iteration(V0,tol=10e-6)
print("The time difference is :", timeit.default_timer() - start_time)

Error at iteration 50 is 0.15564094679169926.
Error at iteration 100 is 0.011973225541353827.
Error at iteration 150 is 0.0009212795432702592.
Error at iteration 200 is 7.088783168285318e-05.

Converged in 239 iterations.
The time difference is : 0.2425708000000668


In [19]:
# second search
import timeit
import numpy as np
import matplotlib.pyplot as plt
beta = 0.95
alpha = 1 / 3
gamma = 1.5
delta = 0.1
kss = ((1 - beta * (1-delta)) / (alpha * beta)) ** (1 / (alpha - 1))
kmax = 1.5 * kss
kmin = 0.5 * kss
n=1000
kgrid = np.linspace(kmin, kmax, n)
# transform V0
from scipy.optimize import minimize_scalar
V0=np.interp(kgrid,np.linspace(kmin, kmax, 100),V)
# step 2
@njit
def V_update(V):
    global gamma
    global delta
    V_new = np.zeros(n)  # initialize updated value function and policy function
    g_new = np.zeros(n)
    for i in range(n):  # loop over grid points
        k = kgrid[i]
        k_bound = k**alpha+(1-delta)*k  # keep non-zero consumption
        V_max = -np.inf  # initialize maximum value
        for j in range(n):  # loop over possible k_next
            k_next = kgrid[j]
            V_next = V[j]
            if k_next < k_bound:  # check feasibility
                c = k**alpha - k_next + (1-delta)*k
                V_current = (c**(1-gamma))/(1-gamma)+beta*V_next
                if V_current > V_max:
                    V_max = V_current
                    g_k = k_next
        V_new[i] = V_max  # save
        g_new[i] = g_k
    return V_new, g_new
# step 3
def V_iteration(V_initial,tol):
    V = V_initial
    error = np.inf
    count = 0
    max_iter = 1000
    print_skip = 50
    while count < max_iter and error > tol:
        V_new, g_new = V_update(V)
        error = np.max(np.abs(V_new - V))
        V = V_new
        count = count + 1
        if count % print_skip == 0:
            print(f"Error at iteration {count} is {error}.")
    if error > tol:
        print("Failed to converge!")
    else:
        print(f"\nConverged in {count} iterations.")
    return V_new, g_new
start_time = timeit.default_timer()
V, g = V_iteration(V0,tol=10e-6)
print("The time difference is :", timeit.default_timer() - start_time)


Converged in 31 iterations.
The time difference is : 1.11374509999996


## 总结

下面我将所有运行结果汇总为一张表格：

| 优化手段 | 迭代次数 | 运行速度 |
| ---- | ---- | ---- |
| **without compiling** |  |  |
| benchmark code | 239 | 286.199236 |
| concave value function only | 239 | 179.71150850000004 |
| increasing policy function only | 239 | 124.27733249999983 |
| both | 239 | 1.2397132000005513 |
| multigrid | 239+31 | 2.9914582000001246+43.685797699999966 |
| **with compiling** |  |  |
| benchmark code | 239 | 6.652758100000028 |
| concave value function only | 239 | 4.2079571999997825 |
| increasing policy function only | 239 | 2.6339558000001944 |
| both | 239 | 0.37810940000008486 |
| multigrid | 239+31 | 0.2425708000000668+1.11374509999996 |


这个表格展示了不同优化手段在两种情况下的运行速度。它包括了“without compiling”和“with compiling”两种场景下的数据。每种场景下又分为四种不同的优化手段，分别是：“benchmark code”、“concave value function only”、“increasing policy function only”以及“both”。对于每种优化手段，都给出了其在两种场景下的迭代次数和运行速度。可以看出编译对于运行速度的提升是非常显著且十分容易实现的；但是我们也不能忽视从理论角度优化算法对应的威力：仅仅通过理论减少不必要的运算也可以实现一秒级的运算速度！